# <center> Data Preparation </center>

## Basic ToolBox - Preparação de Dados III

In [62]:
#!pip install category-encoders
#!pip install  feature-engine
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder
from category_encoders import TargetEncoder
#from feature_engine.categorical_encoders import CountFrequencyCategoricalEncoder
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [63]:
# Carregando DataSet para exemplificação
df_titanic = pd.read_csv('https://raw.githubusercontent.com/abnr/ml-data/main/titanic_reduced.csv')
display(df_titanic.head())

,Embarked,Age,Sex,Pclass,Survived
0,S,22.0,male,3rd,0
1,C,38.0,female,1st,1
2,S,26.0,female,3rd,1
3,S,35.0,female,1st,1
4,S,35.0,male,3rd,0


In [64]:
df_titanic.shape

(891, 5)

In [65]:
# Checando a existencia de valores ausentes, nulos
df_titanic.isna().sum()

Embarked      2
Age         177
Sex           0
Pclass        0
Survived      0
dtype: int64

In [66]:
# removendo dados ausentes, nulos
df_titanic.dropna(inplace=True)

In [67]:
df_titanic.shape

(712, 5)

In [68]:
# Variaveis Independentes
X_titanic = df_titanic.drop('Survived', axis=1)

# Variavel Dependente
Y_titanic = df_titanic['Survived']

In [69]:
# Divivdindo o dataset em treino e teste
X_train_titanic, X_test_titanic, Y_train_titanic, Y_test_titanic = train_test_split(X_titanic,Y_titanic,
                                                                                        train_size=0.85,
                                                                                        random_state=123)

In [84]:
Y_test_titanic.value_counts(normalize=True)

Survived
0    0.551402
1    0.448598
Name: proportion, dtype: float64

Existem no DataSet as variáveis:

- **Embarked**: Qualitativa Nominal
- **Sex**: Qualitativa Nominal
- **Age**: Numérica Contínua
- **Pclass**: Qualitativa Ordinal
- **Survived**: Target Qualitativo Binário

Desta forma, utilizaremos os seguintes transformadores:

- **OrdinalEncoder**: Pclass
- **MinMaxScaler**: Age
- **OneHotEncoder**: Sex, Embarked

Para **construir** nosso **transformador universal** precisamos **criar tuplas**. O **primeiro elemento** da **tupla** será um **identificador**, uma **string**, que **identifica o nome do processo** (pode ser qualquer nome). 

O **segundo elemento** é o **transformador** em si e suas propriedades, se houverem. O **terceiro elemento** é uma **lista** com o **nome das colunas** que irão **receber o processamento**.

# <center> Column Transformer </center>

Deve se **preparar os dados brutos** utilizando **transformações**, aplicadas a estes mesmos dados, **antes de ajustar um modelo** de aprendizado de máquina.Isso é necessário para garantir que se obtenha uma melhor estruturação do seu problema de modelagem preditiva aos algoritmos de aprendizagem.

Aplicar **transformações de dados como dimensionamento** ou **codificação de variáveis ​​categóricas** é simples quando todas as variáveis ​​de entrada são do mesmo tipo. Pode ser um desafio quando se tem um **conjunto de dados** com **tipos mistos** e deseja **aplicar seletivamente transformações de dados** a alguns recursos de entrada, mas não a todos.

Felizmente, a biblioteca de aprendizado de máquina Python **scikit-learn** fornece o ``ColumnTransformer`` que permite **aplicar seletivamente transformações de dados a diferentes colunas** em seu conjunto de dados.

Até agora vimos como aplicar as **transformações em dados categóricos** e **numéricos** separadamente. Teríamos que coletar os resultados de nossas transformações e agrupá-los para formar um conjunto transformado para inserir em um modelo de Machine Learning. Agora iremos aprender a como **realizar todos os processamentos em um único passo**, com o auxílio das Classes ``ColumnsTransformer`` e ``Pipeline``

As transformações de dados podem ser realizadas usando a biblioteca scikit-learn; por exemplo, a **classe SimpleImputer** pode ser usada para **substituir valores ausentes**, a **classe MinMaxScaler** pode ser usada para **dimensionar valores numéricos** e o **OneHotEncoder** pode ser usado para **codificar variáveis ​​​​categóricas**

#### **Para construir o transformador universal precisamos criar tuplas:**

* O primeiro elemento da tupla será um **identificador**, uma **string**, que **identifica o nome do processo** (pode ser qualquer nome). 
* O segundo elemento é o **transformador** em si e **suas propriedades**, se houverem. 
* O terceiro elemento é uma **lista** com o **nome das colunas** que irão **receber o processamento**.

Em resumo, **cada transformador é uma tupla de três elementos**, o que define o nome do transformador, a transformação a ser aplicada e os índices de coluna aos quais aplicá-la. Por exemplo: (**Nome**, **Objeto**, **Colunas**).

**1º Transformador**

In [72]:
# Atribuindo a uma variavei python a tupla para transformador "ordinal_encoder" em "Pclass"
pipe_cat_ordinal = ('ordinal_encoder', OrdinalEncoder(categories=[['1st', '2nd', '3rd']]), ['Pclass'])
# 1º Elemento -  String de Identificação
# 2º Elemento - O transformador "ordinal_encoder" e as respectivas categorias
# 3º Elemento - Variavel que irá receber a transformação

**2º Transformador**

In [73]:
# Atribuindo a uma variavei python a tupla para transformador "one_hot_encoder" em "Sex"  e "Embarked"
pipe_cat_onehot = ('one_hot_encoder', OneHotEncoder(), ['Sex', 'Embarked'])
# 1º Elemento -  String de Identificação
# 2º Elemento - O transformador "one_hot_encoder" 
# 3º Elemento - Variaveis que irá receber a transformação

**3º Transformador**

In [74]:
# Atribuindo a uma variavei python a tupla para transformador "min_max_scaler" em "Age"
pipe_num_min_max = ('min_max_scaler', MinMaxScaler(), ['Age'])
# 1º Elemento -  String de Identificação
# 2º Elemento - O transformador "min_max_scaler" 
# 3º Elemento - Variaveis que irá receber a transformação

**Lista de Todos os Transformadores**

In [75]:
# Atribuindo a uma variavei python uma lista com todo os transformadores
transformers = [pipe_cat_ordinal, pipe_cat_onehot, pipe_num_min_max]

#### sklearn.compose.``ColumnTransforme``

* Aplica transformadores a colunas de um array ou DataFrame do pandas. Este estimador permite que diferentes colunas ou subconjuntos de colunas da entrada sejam transformados separadamente e os recursos gerados por cada transformador serão concatenados para formar um único espaço de recursos. Isto é útil para dados heterogêneos ou colunares, para combinar vários mecanismos de extração de recursos ou transformações em um único transformador.

In [76]:
# Gerando uma objeto unidimensional com todos os transformaçoões e suas repectivas aplicações.
pre_processador = ColumnTransformer(transformers)

In [77]:
# Ajustando todos os transformadores, transformando os dados, e concatenando os resultados.
pre_processador.fit_transform(X_train_titanic)

array([[2.        , 0.        , 1.        , ..., 0.        , 1.        ,
        0.25627127],
       [0.        , 0.        , 1.        , ..., 0.        , 1.        ,
        0.67225514],
       [1.        , 0.        , 1.        , ..., 0.        , 1.        ,
        0.01676541],
       ...,
       [2.        , 0.        , 1.        , ..., 0.        , 1.        ,
        0.10500441],
       [0.        , 0.        , 1.        , ..., 0.        , 1.        ,
        0.81091643],
       [2.        , 0.        , 1.        , ..., 0.        , 1.        ,
        0.23106013]])

Por fim, usaremos a classe **Pipeline** que irá combinar as etapas de pré-processamento e treinamento e previsão

# <center> Função Pipeline Scikit-learn </center>

A **função pipeline** possibilita aplicar uma **sequência de transformadores de dados com um preditor** final opcional. **Pipeline** permite aplicar sequencialmente uma **lista de transformadores para pré-processar os dados** e, se desejar, concluir a sequência com um **preditor final** para **modelagem preditiva**.

As **etapas intermediárias** do pipeline devem ser **transformadas**, ou seja, devem implementar o **fit** com o métodos **transform**. O **estimador final** só precisa ser implementado **fit**. Os **transformadores no pipeline** podem ser **armazenados em cache** usando  o argumentos **memory**. O objetivo do **pipeline é montar várias etapas** que podem ser **validadas cruzadamente** enquanto se definem parâmetros diferentes.

#### sklearn.pipeline.``Pipeline``

* Pipeline de transformações com um estimador final. Aplique sequencialmente uma lista de transformações e um estimador final

**Parâmetros :**

Uma **Lista de tuplas** (**nome da etapa**, **estimador**) que devem ser encadeadas em ordem sequencial. Para ser compatível com a API scikit-learn, **todas as etapas** devem definir **fit**.

In [78]:
# Gerando o objeto pipeline de transformaçoes e estimador
model = Pipeline(steps=[('pre_processador',pre_processador),
                        ('reglog', LogisticRegression(random_state=123))])# Step - 2
# Exemplo: Regressão Logistica

Para essa classe, passamos uma **lista de tuplas**. Sempre o **modelo vem por último**!! **Antes** dele, pode vir **qualquer transformador**, de qualquer classe que possua o **método fit** e **transform**.

Agora **passar para o pipeline** o **pre_processor** que acabou de ser criado. Esse será o **step número 1**, seguido do **step número 2**, que será o **modelo de Regressão Logística**.

In [79]:
# Step - 1 Ajustando os dados para o "Objeto de Transfortmadores" 
# Step - 2 Treinando o "Modelo de Regressão logistica"
model.fit(X_train_titanic, Y_train_titanic)

Pipeline(steps=[('pre_processador',
                 ColumnTransformer(transformers=[('ordinal_encoder',
                                                  OrdinalEncoder(categories=[['1st',
                                                                              '2nd',
                                                                              '3rd']]),
                                                  ['Pclass']),
                                                 ('one_hot_encoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked']),
                                                 ('min_max_scaler',
                                                  MinMaxScaler(), ['Age'])])),
                ('reglog', LogisticRegression(random_state=123))])

**Previsões do Modelo**

In [80]:
# Inserindo os dados de teste no modelo treinado
predict_ = model.predict(X_test_titanic)

In [83]:
(Y_test_titanic == predict_).value_counts(normalize=True)

Survived
True     0.775701
False    0.224299
Name: proportion, dtype: float64

In [85]:
df_titanic['Survived'].value_counts(normalize=True)

Survived
0    0.595506
1    0.404494
Name: proportion, dtype: float64